Conceptual issues for LTCM

1) LTCM focused on securities such as fixed-income, derivatives, and equities, where there was liquidity and measurable pricing mechanisms. They foucsed on medium to long term trades, often with a timeline of 6 months to 2 years. They favored generating small, consistent wins thorugh their strategies, amplifying their wins with leverage. Finally, they relied on both quantitative and qualitative analysis to forecast market dynamics and find market inefficiencies.

2) LTCM has a lot of top tier talent and proprietary knowledge, which led to development of advanced quantitative models and a technical edge. Additionally, they have favorable finance, which they further utilize by exercising strategic leverage and effective risk management techniques.

3) LTCM managed funding risks in a variety of ways. For haircut risks, they used collateral policies and stress testing. For repo maturity risks, they extended maturities and used liquid assets. For equit redemption risks, they utilized lock-ups and staggered withdrawls. Finally, for loan access, they had favorable credit terms and long term financing.

4) LTCM acounted for liquidity risks with quantitative measurements through VaR modeling, stress testing/scenario analysis, conservative position sizing, and long term financing strategies.

5) Yes, given they operate with leverage ratios between 19:1 to 31:1. This exposed them greatly to systemic shocks and liqudity crises.

6) Convergence trades carry significant risks due to liquidity/funding hallenges during divergence, uncertainty in timing of convergence, extreme market events that could cause model breakdowns, and magnification of losses due to leverage. In short, convergence trades yield significant risk when markets are not stable.

LTCM Risk Decomposition

1)

In [289]:
import numpy as np
import pandas as pd
import portfolio_management_helper as pmh
import statsmodels.api as sm

ltcm_rts = pd.read_excel("../data/ltcm_exhibits_data.xlsx",sheet_name="Exhibit 2")
ltcm_rts = ltcm_rts.iloc[3:-4,[0,2,3]]
ltcm_rts.columns = ['Date','LTCM Gross', 'LTCM Net']
ltcm_rts.set_index(ltcm_rts.columns[0], inplace=True)
ltcm_gross_rts = ltcm_rts[["LTCM Gross"]]
ltcm_net_rts = ltcm_rts[["LTCM Net"]]

sp_rts = pd.read_excel("../data/gmo_analysis_data.xlsx",sheet_name="total returns").set_index("date")[["SPY"]]

In [290]:
rts = [ltcm_gross_rts,ltcm_net_rts,sp_rts]
summary_stats = []

for rt in rts:
    stats = pmh.calc_summary_statistics(
        returns=rt,
        annual_factor=12,
        provided_excess_returns=True,
        keep_columns=["Annualized Mean", "Annualized Vol", "Annualized Sharpe","Skewness","Kurtosis","Annualized Historical VaR"],
    )
    
    summary_stats.append(stats)

comparison_df = pd.concat(summary_stats)
comparison_df

,Annualized Mean,Annualized Vol,Annualized Sharpe,Skewness,Excess Kurtosis,Annualized Historical VaR (5.00%)
LTCM Gross,0.2939,0.1364,2.1553,-0.2964,1.5694,-0.0915
LTCM Net,0.2072,0.1119,1.8513,-0.8179,2.9055,-0.0776
SPY,0.1104,0.1482,0.7451,-0.5908,0.9939,-0.2411


The LTCM instruments significantly outperfrom SPY. The Sharpe ratio is dramatically higher, while the VaR is substantially lower. LTCM does feature Excess Kurtosis and a more negative skewness for net LTCM, which means there could be some more extreme movements, but the lower VaR implies these are mitigated.

2)

In [291]:
sp_rts_filtered = sp_rts[(sp_rts.index > '1994-02-28') & (sp_rts.index < '1998-07-31')]
ltcm_net_rts_filtered = ltcm_net_rts[:-1]
ltcm_net_rts_filtered = ltcm_net_rts_filtered.astype(float)

In [292]:
sp_rts_filtered.head()

,SPY
date,
1994-03-31,-0.0419
1994-04-30,0.0112
1994-05-31,0.0159
1994-06-30,-0.0229
1994-07-31,0.0323


In [293]:
ltcm_net_rts_filtered.head()

,LTCM Net
Date,
1994-03-01,-0.0130
1994-04-01,0.0080
1994-05-01,0.0530
1994-06-01,-0.0290
1994-07-01,0.0840


In [294]:
pmh.calc_regression(
    y = ltcm_net_rts_filtered[["LTCM Net"]],
    X = sp_rts_filtered[["SPY"]],
    annual_factor=12,
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios
"LTCM Net" Required to reset indexes to make regression work. Try passing "y" and "X" as pd.DataFrame


,Annualized Alpha,R-Squared,SPY Beta
LTCM Net,0.1805,0.0173,0.1313


From this regression, it appears that LTCM is not a closet indexer. It has a low R^2, and SPYs Beta is low.

LTCM does seem to deliver excess returns beyond the risk premium expected from market exposure. We can see this in the alpha when we regressed LTCM on SPY, which is .18, meaning there is significant adjustment needed to explain the correlation between the two, highlighting the uncorrelated excess return we get from LTCM.

3)

In [295]:
sp_rts_filtered["SPY Squared"] = sp_rts_filtered["SPY"] ** 2

sp_rts_filtered.head()

,SPY,SPY Squared
date,,
1994-03-31,-0.0419,0.0018
1994-04-30,0.0112,0.0001
1994-05-31,0.0159,0.0003
1994-06-30,-0.0229,0.0005
1994-07-31,0.0323,0.0010


In [296]:
pmh.calc_regression(
    y = ltcm_net_rts_filtered[["LTCM Net"]],
    X = sp_rts_filtered[["SPY","SPY Squared"]],
    annual_factor=12,
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios
"LTCM Net" Required to reset indexes to make regression work. Try passing "y" and "X" as pd.DataFrame


,Annualized Alpha,R-Squared,SPY Beta,SPY Squared Beta
LTCM Net,0.2051,0.0240,0.1818,-2.1336


The quadratic market factor improves the R-squared, but the R-squared is still quite low, showing we still cannot explain LTCM's returns very well.

The beta for SPY Squared is negative, which implies LTCMs returns decrease when market volatility increases, indicating they are short market options. This is because options tend to result in losses during high volatility periods.

This means we can describe LTCM as being negatively exposed to market volatility.

4)

In [297]:
k1 = .03
k2 = -.03

sp_rts_filtered["Up Market"] = np.maximum(sp_rts_filtered["SPY"] - k1,0)
sp_rts_filtered["Down Market"] = np.maximum(k2 - sp_rts_filtered["SPY"],0)

sp_rts_filtered.head()

,SPY,SPY Squared,Up Market,Down Market
date,,,,
1994-03-31,-0.0419,0.0018,0.0000,0.0119
1994-04-30,0.0112,0.0001,0.0000,0.0000
1994-05-31,0.0159,0.0003,0.0000,0.0000
1994-06-30,-0.0229,0.0005,0.0000,0.0000
1994-07-31,0.0323,0.0010,0.0023,0.0000


In [298]:
pmh.calc_regression(
    y = ltcm_net_rts_filtered[["LTCM Net"]],
    X = sp_rts_filtered[["SPY","Up Market","Down Market"]],
    annual_factor=12,
    keep_columns=["Annualized Alpha", "Beta", "R-Squared"],
)

"calc_regression" assumes excess returns to calculate Information and Treynor Ratios
"LTCM Net" Required to reset indexes to make regression work. Try passing "y" and "X" as pd.DataFrame


,Annualized Alpha,R-Squared,SPY Beta,Up Market Beta,Down Market Beta
LTCM Net,0.1408,0.0504,0.4867,-0.7543,1.5220


LTCM is short the call-like factor, as it has a negative beta. However, it is long the put-like factor, as this factor has a positive beta.

The put-like factor moves LTCM more, seeing as it has a larger beta.

LTCM's volatility exposure comes from being short the market upside, and long the market downside, as indicated by the betas. Together, these indicate LTCM's strategy is aligned with profiting in declining markets and losing with rising markets, which is a short volatility exposure.

 The FX Carry Trade